In [1]:
from huggingface_hub import login
login()

In [21]:
from datasets import load_dataset
import json
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os


In [30]:
ds = load_dataset("cat-searcher/minif2f-lean4", split="test")
save_path = os.path.expanduser("~/assip2025/data/minif2f_saved")
ds.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/244 [00:00<?, ? examples/s]

In [31]:
save_path = os.path.expanduser("~/assip2025/data/minif2f_saved")
from datasets import load_from_disk
ds = load_from_disk(save_path)

In [23]:
MODEL_ID = "RickyDeSkywalker/TheoremLlama"
tok  = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto"          
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [26]:
model_save_path = os.path.expanduser("~/assip2025/saved_models/saved_theorem_llama")
tok.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

/root/assip2025/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3737: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [27]:
model_save_path = os.path.expanduser("~/assip2025/saved_models/saved_theorem_llama")

tok = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForCausalLM.from_pretrained(model_save_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [28]:
streamer = TextStreamer(tok)

In [36]:
PROOF_DIR    = os.path.expanduser("~/assip2025/llama_proofs/LlamaProofs")
SAVE_DIR = os.path.expanduser("~/assip2025/llama_proofs/LlamaProofs")

In [ ]:
for ex in ds:
    pid = ex["id"]                  
    lean_out = f"{PROOF_DIR}/{pid}.lean"
    if os.path.exists(lean_out):
        print(f"✔︎ {pid} already done, skipping.")
        continue


    header   = ex.get("header", "")
    theorem  = ex["formal_statement"]
    nl_desc  = ex["informal_stmt"]

    prompt = (
        f"-- Informal description\n{nl_desc}\n\n"
        f"-- Formal theorem skeleton\n{theorem}\n\n"
        f"-- Fill in Lean proof after `:= by` or `:=\nby`:\n"
    )

    input_ids = tok(prompt, return_tensors="pt").to(model.device)

    gen_ids = model.generate(
        **input_ids,
        max_new_tokens=512,
        do_sample=False,      # greedy; raise for exploration
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id,
        streamer=streamer,
    )
    proof = tok.decode(gen_ids[0][input_ids["input_ids"].shape[1]:],
                       skip_special_tokens=True)

    # ===== 6. Save .lean file =====
    with open(lean_out, "w") as f:
        f.write(f"{header}\n\n{theorem[:-6]}:= by\n{proof}\n")
    print(f"💾  Saved proof for {pid} → {lean_out}")

In [35]:
tok.pad_token = tok.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [37]:

if __name__ == "__main__":
    total_problems = len(ds)
    print(f"\nStarting ZERO-SHOT proof generation for {total_problems} problems...")

    for i, problem in enumerate(ds):
        problem_name = problem.get("name")
        prompt = problem.get("theorem")

        if not problem_name or not prompt:
            print(f"Warning: Skipping problem at index {i} due to missing name or theorem.")
            continue

        print(f"\n[{i+1}/{total_problems}] Processing Problem: {problem_name}")

       
        inputs = tok(prompt, return_tensors="pt").to(model.device)

        #    max_new_tokens controls how long the proof can be. Adjust as needed.
        #    temperature=0 means the model will be deterministic (less creative).
        #    For classification of common errors, a low temperature is good.
        print("  Generating proof...")
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=True, # Must be True for temperature to have an effect
            top_k=50,
            top_p=0.95
        )

        # 3. Decode the generated tokens back to text
        #    We skip special tokens (like padding) and decode only the newly generated part.
        completion = tok.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

        # Combine the original prompt with the model's completion
        full_proof_attempt = prompt + completion
        # --- Save the proof to a .lean file ---
        output_filename = os.path.join(SAVE_DIR, f"{problem_name}.lean")
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                f.write(full_proof_attempt)
            print(f"  Proof attempt saved to: {output_filename}")
        except IOError as e:
            print(f"  Error saving file {output_filename}: {e}")

    print(f"\n--- Generation Complete ---")
    print(f"All {total_problems} proof attempts have been generated and saved in '{SAVE_DIR}'.")


Starting ZERO-SHOT proof generation for 244 problems...

--- Generation Complete ---
All 244 proof attempts have been generated and saved in '/root/assip2025/llama_proofs/LlamaProofs'.
